In [1]:
# Load necessary packages
library(RNifti)
library(dplyr)
library(oro.nifti)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


oro.nifti 0.11.4


Attaching package: ‘oro.nifti’


The following object is masked from ‘package:dplyr’:

    slice


The following object is masked from ‘package:RNifti’:

    origin




In [2]:
# Create necessary data frames
results_dir <- "scratch/tractoflow_hcp_dwi/results/"
patients <- list.files(results_dir, full.names = TRUE)
patients <- patients[grep("^\\d{6}$", basename(patients))]
patient_data <- read.csv("scratch/tractoflow_hcp_dwi/patient_data.csv")
patient_data_gender <- read.csv("scratch/tractoflow_hcp_dwi/patient_data_gender.csv")
subj_data <- patient_data[c("Subject", "Age_in_Yrs", "BMI", "SSAGA_Income", "SSAGA_Educ", "BPSystolic", "BPDiastolic", "HbA1C")]
subj_data_gender <- patient_data_gender[c("Subject", "Gender")]
subj_data=merge(subj_data, subj_data_gender)

# Define variable to extract hypothalamic values
rois <- c("caudate_spm", "hippocampus_spm", "nacc_spm", "pallidum_spm", "putamen_spm")

In [3]:
# Create empty data frame
roi_data <- data.frame(Subject=character(0))

# Loop through each patient
for (i in 1:length(patients)) {
    # Define necessary lists
    var_names <-list()
    data <-list()
    avg_vals <- list()
    
    # Loop through each ROI
    for (roi in 1:length(rois)) {
        # Predefining the first column values
        if (i==1) {
            roi_data[[rois[roi]]] <- numeric(0)
        }
        # Extract ROI values and define variable with only ROI NIfTI values above 0
        var_names[[roi]] <- file.path("scratch", "tractoflow_hcp_dwi", "subject_other_rois", basename(patients[i]), "ad_rois", paste0(basename(patients[i]), "_ad_", rois[roi], "_bin_resampled.nii.gz"))
        data[[roi]] <- readNifti(var_names[[roi]])
        non_zero_vals <- data[[roi]]
        non_zero_vals <- non_zero_vals[non_zero_vals > 0]
        # Calculate ROI average value and append it to the data frame
        avg_val <- mean(non_zero_vals)
        avg_vals <- append(avg_vals, avg_val)
        temp_data <- data.frame(Subject=basename(patients[i]), avg_vals)
    }
    # Define column names and bind data calculated in the for loop
    colnames(temp_data)=c("Subject", rois)
    roi_data <- rbind(roi_data, temp_data)
}
# Merge hypothalamic values with patient data and write as CSV
subj_data <- merge(subj_data, roi_data)
write.csv(subj_data, "scratch/tractoflow_hcp_dwi/linear_regression/ad_other_rois.csv")